In [78]:
from __future__ import print_function
import torch.utils.data as data
import os
import os.path
import torch
import numpy as np
import sys
from tqdm import tqdm 
import json
from plyfile import PlyData, PlyElement

In [79]:
root = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0'

In [80]:
catfile = os.path.join(root, 'synsetoffset2category.txt')
cat = {}
meta = {}

In [81]:
with open(catfile, 'r') as f:
    for line in f:
        ls = line.strip().split()
        cat[ls[0]] = ls[1]

In [82]:
for item in cat:
    dir_seg = os.path.join(root, cat[item], 'points_label')
    dir_point = os.path.join(root, cat[item], 'points')
    fns = sorted(os.listdir(dir_point))
    meta[item] = []
    for fn in fns:
        token = (os.path.splitext(os.path.basename(fn))[0])
        meta[item].append((os.path.join(dir_point, token + '.pts'), os.path.join(dir_seg, token + '.seg')))

In [86]:
print(meta.items())[0]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: 'NoneType' object is not subscriptable

In [23]:
with open(os.path.join(os.getcwd(), '../misc/num_seg_classes_test.txt'), 'w') as f:
    for item in cat:
        datapath = []
        num_seg_classes = 0
        for fn in meta[item]:
            datapath.append((item, fn[0], fn[1]))

        for i in tqdm(range(len(datapath))):
            l = len(np.unique(np.loadtxt(datapath[i][-1]).astype(np.uint8)))
            if l > num_seg_classes:
                num_seg_classes = l

        print("category {} num segmentation classes {}".format(item, num_seg_classes))
        f.write("{}\t{}\n".format(item, num_seg_classes))

100%|██████████| 2690/2690 [00:09<00:00, 271.67it/s]


category Airplane num segmentation classes 4


100%|██████████| 76/76 [00:00<00:00, 252.77it/s]


category Bag num segmentation classes 2


100%|██████████| 55/55 [00:00<00:00, 267.40it/s]


...

category Rocket num segmentation classes 3


100%|██████████| 152/152 [00:00<00:00, 265.39it/s]


category Skateboard num segmentation classes 3


100%|██████████| 5266/5266 [00:20<00:00, 258.17it/s]

category Table num segmentation classes 3


In [33]:
npoints = 2500
root = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0'
cat = {}
data_augmentation = True
classification = False
split = 'train'
seg_classes = {}
class_choice=None

In [34]:
catfile = os.path.join(root, 'synsetoffset2category.txt')

with open(catfile, 'r') as f:
    for line in f:
        ls = line.strip().split()
        cat[ls[0]] = ls[1]


In [35]:
if not class_choice is None:
    cat = {k: v for k, v in cat.items() if k in class_choice}

In [37]:
print(cat)

{'Airplane': '02691156', 'Bag': '02773838', 'Cap': '02954340', 'Car': '02958343', 'Chair': '03001627', 'Earphone': '03261776', 'Guitar': '03467517', 'Knife': '03624134', 'Lamp': '03636649', 'Laptop': '03642806', 'Motorbike': '03790512', 'Mug': '03797390', 'Pistol': '03948459', 'Rocket': '04099429', 'Skateboard': '04225987', 'Table': '04379243'}


In [38]:
id2cat = {v: k for k, v in cat.items()}
print(id2cat)

{'02691156': 'Airplane', '02773838': 'Bag', '02954340': 'Cap', '02958343': 'Car', '03001627': 'Chair', '03261776': 'Earphone', '03467517': 'Guitar', '03624134': 'Knife', '03636649': 'Lamp', '03642806': 'Laptop', '03790512': 'Motorbike', '03797390': 'Mug', '03948459': 'Pistol', '04099429': 'Rocket', '04225987': 'Skateboard', '04379243': 'Table'}


In [39]:
meta = {}
splitfile = os.path.join(root, 'train_test_split', 'shuffled_{}_file_list.json'.format(split))

In [40]:
filelist = json.load(open(splitfile, 'r'))
for item in cat:
    meta[item] = []

In [41]:
print(meta)

{'Airplane': [], 'Bag': [], 'Cap': [], 'Car': [], 'Chair': [], 'Earphone': [], 'Guitar': [], 'Knife': [], 'Lamp': [], 'Laptop': [], 'Motorbike': [], 'Mug': [], 'Pistol': [], 'Rocket': [], 'Skateboard': [], 'Table': []}


In [44]:
for file in filelist:
    _, category, uuid = file.split('/')
    if category in cat.values():
        meta[id2cat[category]].append((os.path.join(root, category, 'points', uuid+'.pts'),
                                os.path.join(root, category, 'points_label', uuid+'.seg')))

In [58]:
list(meta.values())[0], list(meta.keys())[0]

([('/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points/ed73e946a138f3cfbc0909d98a1ff2b4.pts',
   '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points_label/ed73e946a138f3cfbc0909d98a1ff2b4.seg'),
  ('/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points/68537bfb054ee3802ffb52751fee2c0d.pts',
   '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points_label/68537bfb054ee3802ffb52751fee2c0d.seg'),
  ('/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benc

In [60]:
datapath = []
for item in cat:
    for fn in meta[item]:
        datapath.append((item, fn[0], fn[1]))

In [64]:
print(len(datapath))

12137


In [65]:
print(datapath[0])

('Airplane', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points/ed73e946a138f3cfbc0909d98a1ff2b4.pts', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0/02691156/points_label/ed73e946a138f3cfbc0909d98a1ff2b4.seg')


In [66]:
classes = dict(zip(sorted(cat), range(len(cat))))
print(classes)

{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}


In [67]:
with open(os.path.join(os.getcwd(), '../misc/num_seg_classes_test.txt'), 'r') as f:
    for line in f:
        ls = line.strip().split()
        seg_classes[ls[0]] = int(ls[1])
num_seg_classes = seg_classes[list(cat.keys())[0]]
print(seg_classes, num_seg_classes)

{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4


In [ ]:
##################################################################################################3

In [70]:
index = 0

In [71]:
fn = datapath[index]
cls = classes[datapath[index][0]]
point_set = np.loadtxt(fn[1]).astype(np.float32)
seg = np.loadtxt(fn[2]).astype(np.int64)

In [72]:
print(point_set, cls, seg)

[[-0.01449 -0.01636  0.03202]
 [-0.01188 -0.06567  0.01454]
 [-0.14239 -0.03697 -0.05186]
 ...
 [ 0.03415 -0.09312 -0.05231]
 [-0.0108  -0.06001  0.05221]
 [-0.01654 -0.05931  0.05599]] 0 [1 1 4 ... 4 2 2]


In [73]:
print(point_set.shape, seg.shape)

(2518, 3) (2518,)


In [74]:
choice = np.random.choice(len(seg), npoints, replace=True)
# resample
point_set = point_set[choice, :]

point_set = point_set - np.expand_dims(np.mean(point_set, axis = 0), 0) # center
dist = np.max(np.sqrt(np.sum(point_set ** 2, axis = 1)),0)
point_set = point_set / dist # scale

In [75]:
print(point_set)

[[ 0.06521711 -0.05850023 -0.34190094]
 [-0.28736272 -0.0049937  -0.03989419]
 [ 0.0374782  -0.0641325  -0.27536985]
 ...
 [ 0.13031672 -0.08454947  0.1257883 ]
 [-0.7445385   0.25648427 -0.03630362]
 [ 0.2430794   0.07507835 -0.01952416]]


In [76]:
seg = seg[choice]
point_set = torch.from_numpy(point_set)
seg = torch.from_numpy(seg)
cls = torch.from_numpy(np.array([cls]).astype(np.int64))

In [77]:
print(point_set, cls, seg)

tensor([[ 0.0652, -0.0585, -0.3419],
        [-0.2874, -0.0050, -0.0399],
        [ 0.0375, -0.0641, -0.2754],
        ...,
        [ 0.1303, -0.0845,  0.1258],
        [-0.7445,  0.2565, -0.0363],
        [ 0.2431,  0.0751, -0.0195]]) tensor([0]) tensor([2, 1, 2,  ..., 1, 3, 1])
